# Big Data Project Notebook

## Dataset

Name: NYC Taxi & Limousine Commission (TLC) Trip Record Data
Source link: [NYC TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page?utm_source=chatgpt.com)

## Main Job

**Objective**: Analyze how tip generosity varies by pickup location and time of day, and identify the top pickup zones with the most generous passengers.

### Plan

#### Setup

- clean up and select relevant columns
- create derived columns (tip percentage, hour of day)

#### Shuffles

- join zones id with zone lookup table
- aggregate per location and hour
- aggregate per average tip percentage
- order by average tip percentage


# Setup

- Import libraries
- Setup Spark
- Load dataset in memory

In [1]:
# Import libraries
import findspark
from pyspark.sql import SparkSession

# Setup Spark
findspark.init()
spark = SparkSession.builder \
    .appName("NYC Taxi Analysis - Sample") \
    .getOrCreate()
## health check
df = spark.range(5)
df.show()

# Load sample data
trip_file = "sample_data/yellow_tripdata_2025-01.parquet"
df_trips = spark.read.parquet(trip_file)
## health check
df_trips.show(5)
df_trips.printSchema()
# Load zone lookup table
zone_file = "sample_data/taxi_zone_lookup.csv"
df_zones = spark.read.csv(zone_file, header=True, inferSchema=True)
## health check
df_zones.show(5)
df_zones.printSchema()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/30 18:18:12 WARN Utils: Your hostname, rioly, resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlan0)
25/10/30 18:18:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/30 18:18:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|      

# Main Job

## Join Trip Data with Zone Lookup Table

In [2]:
from pyspark.sql.functions import col

df_joined = df_trips.join(
    df_zones,
    df_trips.PULocationID == df_zones.LocationID,
    "left"
)

# Select relevant columns only
df_joined = df_joined.select(
    "PULocationID",
    "Borough",
    "Zone",
    "tpep_pickup_datetime",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "payment_type"
)

# Health check
df_joined.show(5)
print("Joined dataset count:", df_joined.count())


+------------+---------+--------------------+--------------------+-------------+-----------+----------+------------+------------+
|PULocationID|  Borough|                Zone|tpep_pickup_datetime|trip_distance|fare_amount|tip_amount|total_amount|payment_type|
+------------+---------+--------------------+--------------------+-------------+-----------+----------+------------+------------+
|         229|Manhattan|Sutton Place/Turt...| 2025-01-01 00:18:38|          1.6|       10.0|       3.0|        18.0|           1|
|         236|Manhattan|Upper East Side N...| 2025-01-01 00:32:40|          0.5|        5.1|      2.02|       12.12|           1|
|         141|Manhattan|     Lenox Hill West| 2025-01-01 00:44:04|          0.6|        5.1|       2.0|        12.1|           1|
|         244|Manhattan|Washington Height...| 2025-01-01 00:14:27|         0.52|        7.2|       0.0|         9.7|           2|
|         244|Manhattan|Washington Height...| 2025-01-01 00:21:34|         0.66|        5.

## Compute derived columns

In [3]:
from pyspark.sql.functions import col, hour, round

# Filter out trips with zero fare to avoid division by zero
df_cleaned = df_joined.filter(col("fare_amount") > 0)

df_enriched = df_cleaned.withColumn(
    "tip_percentage", round(col("tip_amount") / col("fare_amount") * 100, 2)
).withColumn(
    "hour_of_day", hour(col("tpep_pickup_datetime"))
)

# Health check
df_enriched.select("PULocationID", "Borough", "Zone", "hour_of_day", "tip_percentage").show(5)


+------------+---------+--------------------+-----------+--------------+
|PULocationID|  Borough|                Zone|hour_of_day|tip_percentage|
+------------+---------+--------------------+-----------+--------------+
|         229|Manhattan|Sutton Place/Turt...|          0|          30.0|
|         236|Manhattan|Upper East Side N...|          0|         39.61|
|         141|Manhattan|     Lenox Hill West|          0|         39.22|
|         244|Manhattan|Washington Height...|          0|           0.0|
|         244|Manhattan|Washington Height...|          0|           0.0|
+------------+---------+--------------------+-----------+--------------+
only showing top 5 rows


## Aggregate by Pickup Zone and Hour

In [4]:
from pyspark.sql.functions import avg, count

df_agg_hour = df_enriched.groupBy(
    "PULocationID", "Borough", "Zone", "hour_of_day"
).agg(
    round(avg("tip_percentage"), 2).alias("avg_tip_pct"),
    count("*").alias("num_trips")
)

# Health check
df_agg_hour.show(10)


[Stage 18:============================================>           (12 + 3) / 15]

+------------+---------+--------------------+-----------+-----------+---------+
|PULocationID|  Borough|                Zone|hour_of_day|avg_tip_pct|num_trips|
+------------+---------+--------------------+-----------+-----------+---------+
|         229|Manhattan|Sutton Place/Turt...|          1|      20.45|      669|
|         265|      N/A|      Outside of NYC|          1|      29.67|       55|
|         152|Manhattan|      Manhattanville|          0|       4.78|       51|
|         211|Manhattan|                SoHo|          2|      16.76|      773|
|         256| Brooklyn|Williamsburg (Sou...|         18|       4.91|      111|
|         243|Manhattan|Washington Height...|         22|        2.2|       50|
|          51|    Bronx|          Co-Op City|         15|        0.0|       14|
|         243|Manhattan|Washington Height...|         16|       3.92|       39|
|          53|   Queens|       College Point|         20|        0.0|        8|
|         159|    Bronx|       Melrose S

## Aggregate Across Hours per Pickup Zone

In [9]:
from pyspark.sql.functions import avg, round, desc, sum as spark_sum

df_agg_zone = df_agg_hour.groupBy(
    "PULocationID", "Borough", "Zone"
).agg(
    round(avg("avg_tip_pct"), 2).alias("avg_tip_pct_overall"),
    spark_sum("num_trips").alias("total_trips")
)

# Health check
df_agg_zone.show(10)

[Stage 24:====================================================>   (14 + 1) / 15]

+------------+---------+--------------------+-------------------+-----------+
|PULocationID|  Borough|                Zone|avg_tip_pct_overall|total_trips|
+------------+---------+--------------------+-------------------+-----------+
|         136|    Bronx| Kingsbridge Heights|               0.84|        273|
|         219|   Queens|Springfield Garde...|               6.47|        693|
|          92|   Queens|            Flushing|               0.92|        531|
|          53|   Queens|       College Point|               3.67|        189|
|          94|    Bronx|       Fordham South|                0.0|        157|
|         123| Brooklyn|           Homecrest|               1.65|        366|
|          33| Brooklyn|    Brooklyn Heights|               8.58|       2315|
|         158|Manhattan|Meatpacking/West ...|              17.73|      31669|
|         146|   Queens|Long Island City/...|               7.69|       1738|
|         240|    Bronx|  Van Cortlandt Park|               0.32

## Order by average tip percentage

In [10]:
df_top_zones = df_agg_zone.orderBy(desc("avg_tip_pct_overall"))

# Show top 10 tipping zones
df_top_zones.show(10)

[Stage 31:============================================>           (12 + 3) / 15]

+------------+-------------+--------------------+-------------------+-----------+
|PULocationID|      Borough|                Zone|avg_tip_pct_overall|total_trips|
+------------+-------------+--------------------+-------------------+-----------+
|         265|          N/A|      Outside of NYC|             387.06|       1285|
|          29|     Brooklyn|      Brighton Beach|             204.43|        250|
|          23|Staten Island|Bloomfield/Emerso...|             126.67|         12|
|          22|     Brooklyn|    Bensonhurst West|              55.97|        317|
|           1|          EWR|      Newark Airport|              34.32|        366|
|         199|        Bronx|       Rikers Island|              25.05|          1|
|         264|      Unknown|                 N/A|               23.2|       8040|
|         162|    Manhattan|        Midtown East|              22.95|     114449|
|         107|    Manhattan|            Gramercy|              22.34|      65806|
|         142|  

---

# Optimized Job

## Setup

In [11]:
# Imports
import findspark
from pyspark.sql import SparkSession

# Initialize Spark
findspark.init()

spark = (
    SparkSession.builder
    .appName("NYC Taxi Analysis - Optimized")
    .master("local[*]")  # use all available cores
    .config("spark.sql.shuffle.partitions", 100)
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

# Health check
df_test = spark.range(5)
df_test.show()


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



25/10/30 17:54:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Load dataset

In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

trip_file = "sample_data/yellow_tripdata_2025-01.parquet"
zone_file = "sample_data/taxi_zone_lookup.csv"

# Load trip data
df_trips = (
    spark.read.parquet(trip_file)
    .select(
        "PULocationID",           # key for aggregation
        "tpep_pickup_datetime",   # to extract hour
        "fare_amount",            # for tip percentage
        "tip_amount"              # needed
    )
)

# Cache for reuse
df_trips.cache()
df_trips.count()

# Health check
df_trips.show(5)
df_trips.printSchema()

# Load zone lookup table
zone_schema = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True)
])

df_zones = (
    spark.read.csv(zone_file, header=True, schema=zone_schema)
    .select("LocationID", "Borough", "Zone")
)

df_zones.cache()
df_zones.count()

# Health check
df_zones.show(5)
df_zones.printSchema()


25/10/30 17:57:50 WARN CacheManager: Asked to cache already cached data.


+------------+--------------------+-----------+----------+
|PULocationID|tpep_pickup_datetime|fare_amount|tip_amount|
+------------+--------------------+-----------+----------+
|         229| 2025-01-01 00:18:38|       10.0|       3.0|
|         236| 2025-01-01 00:32:40|        5.1|      2.02|
|         141| 2025-01-01 00:44:04|        5.1|       2.0|
|         244| 2025-01-01 00:14:27|        7.2|       0.0|
|         244| 2025-01-01 00:21:34|        5.8|       0.0|
+------------+--------------------+-----------+----------+
only showing top 5 rows
root
 |-- PULocationID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)

+----------+-------------+--------------------+
|LocationID|      Borough|                Zone|
+----------+-------------+--------------------+
|         1|          EWR|      Newark Airport|
|         2|       Queens|         Jamaica Bay|
|         

25/10/30 17:57:50 WARN CacheManager: Asked to cache already cached data.


## Join Tables

In [14]:
from pyspark.sql.functions import broadcast

# broadcast the small zone lookup table
df_enriched = df_trips.join(
    broadcast(df_zones),
    df_trips.PULocationID == df_zones.LocationID,
    "left"
).select(
    "PULocationID",
    "Borough",
    "Zone",
    "tpep_pickup_datetime",
    "fare_amount",
    "tip_amount"
)

# Cache the joined dataset for reuse
df_enriched.cache()
df_enriched.count()

# Health check
df_enriched.show(5)


+------------+---------+--------------------+--------------------+-----------+----------+
|PULocationID|  Borough|                Zone|tpep_pickup_datetime|fare_amount|tip_amount|
+------------+---------+--------------------+--------------------+-----------+----------+
|         229|Manhattan|Sutton Place/Turt...| 2025-01-01 00:18:38|       10.0|       3.0|
|         236|Manhattan|Upper East Side N...| 2025-01-01 00:32:40|        5.1|      2.02|
|         141|Manhattan|     Lenox Hill West| 2025-01-01 00:44:04|        5.1|       2.0|
|         244|Manhattan|Washington Height...| 2025-01-01 00:14:27|        7.2|       0.0|
|         244|Manhattan|Washington Height...| 2025-01-01 00:21:34|        5.8|       0.0|
+------------+---------+--------------------+--------------------+-----------+----------+
only showing top 5 rows


## Aggregate by Pickup Zone and Hour

In [15]:
from pyspark.sql.functions import col, hour, round, avg, count, desc, sum as spark_sum

# Derived columns
df_derived = df_enriched.filter(col("fare_amount") > 0).withColumn(
    "tip_percentage", round(col("tip_amount") / col("fare_amount") * 100, 2)
).withColumn(
    "hour_of_day", hour(col("tpep_pickup_datetime"))
)

# Cache derived dataset for reuse
df_derived.cache()
df_derived.count()

# Aggregate by pickup zone and hour
df_agg_hour = df_derived.groupBy(
    "PULocationID", "Borough", "Zone", "hour_of_day"
).agg(
    round(avg("tip_percentage"), 2).alias("avg_tip_pct"),
    count("*").alias("num_trips")
)

df_agg_hour.cache()
df_agg_hour.show(10)

[Stage 71:============================================>           (12 + 3) / 15]

+------------+---------+--------------------+-----------+-----------+---------+
|PULocationID|  Borough|                Zone|hour_of_day|avg_tip_pct|num_trips|
+------------+---------+--------------------+-----------+-----------+---------+
|           7|   Queens|             Astoria|          0|       5.94|      100|
|         229|Manhattan|Sutton Place/Turt...|          1|      20.45|      669|
|         265|      N/A|      Outside of NYC|          1|      29.67|       55|
|         152|Manhattan|      Manhattanville|          0|       4.78|       51|
|         211|Manhattan|                SoHo|          2|      16.76|      773|
|          82|   Queens|            Elmhurst|          3|       1.81|       43|
|          90|Manhattan|            Flatiron|          4|      16.79|      450|
|         162|Manhattan|        Midtown East|          8|      23.42|     4453|
|         193|   Queens|Queensbridge/Rave...|          9|       3.61|       97|
|         249|Manhattan|        West Vil

## Aggregate Across Hours

In [16]:
# Aggregate overall per pickup zone and order (second shuffle) ---
df_agg_zone = df_agg_hour.groupBy(
    "PULocationID", "Borough", "Zone"
).agg(
    round(avg("avg_tip_pct"), 2).alias("avg_tip_pct_overall"),
    spark_sum("num_trips").alias("total_trips")
)

# Show top 10 tipping zones
df_top_zones.show(10)

[Stage 77:================================================>       (13 + 2) / 15]

+------------+-------------+--------------------+-------------------+-----------+
|PULocationID|      Borough|                Zone|avg_tip_pct_overall|total_trips|
+------------+-------------+--------------------+-------------------+-----------+
|         265|          N/A|      Outside of NYC|             387.06|       1285|
|          29|     Brooklyn|      Brighton Beach|             204.43|        250|
|          23|Staten Island|Bloomfield/Emerso...|             126.67|         12|
|          22|     Brooklyn|    Bensonhurst West|              55.97|        317|
|           1|          EWR|      Newark Airport|              34.32|        366|
|         199|        Bronx|       Rikers Island|              25.05|          1|
|         264|      Unknown|                 N/A|               23.2|       8040|
|         162|    Manhattan|        Midtown East|              22.95|     114449|
|         107|    Manhattan|            Gramercy|              22.34|      65806|
|         142|  